In [ ]:
import os
import sys
import pickle
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import matplotlib.pyplot as plt

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\WSCC'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Successfully obtained active project.')

In [ ]:
project_folders = {}
for folder_name in ('study',):
    project_folders[folder_name] = app.GetProjectFolder(folder_name)
    if project_folders[folder_name] is None:
        raise Exception(f'No folder "{folder_name}" present')
    print(f'Successfully obtained folder "{folder_name}".')

In [ ]:
sort_fun = lambda x: x.loc_name
generators = sorted(app.GetCalcRelevantObjects('*.ElmSym'), key=sort_fun)
lines = sorted(app.GetCalcRelevantObjects('*.ElmLne'), key=sort_fun)
buses = sorted(app.GetCalcRelevantObjects('*.ElmTerm'), key=sort_fun)
loads = sorted(app.GetCalcRelevantObjects('*.ElmLod'), key=sort_fun)
n_generators, n_lines, n_buses, n_loads = len(generators), len(lines), len(buses), len(loads)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')

In [ ]:
P = {}
S = {}
H = {}
for generator in generators:
    i = int(generator.loc_name[1:])
    generator_type = generator.GetAttribute('typ_id')
    P[i] = generator.GetAttribute('pgini')
    S[i] = generator_type.GetAttribute('sgn')
    H[i] = generator_type.GetAttribute('h')
    print(f'{generator.loc_name}: P = {P[i]:4.0f} MW, S = {S[i]:5.0f} MVA, inertia = {H[i]:5.2f} s.')

In [ ]:
for line in lines:
    line_type = line.GetAttribute('typ_id')
    vrating = line_type.GetAttribute('uline')
    print(f'{line.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
for bus in buses:
    vrating = bus.GetAttribute('uknom')
    print(f'{bus.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
for load in loads:
    plini = load.GetAttribute('plini')
    qlini = load.GetAttribute('qlini')
    print(f'{load.loc_name}: P = {plini:5.1f} MW, Q = {qlini:5.1f} MVAR.')

## Load flow analysis

In [ ]:
def run_load_flow(project_folder, generators, loads, buses, study_case_name='01- Load Flow', verbose=False):
    #
    study_case = project_folder.GetContents(study_case_name)[0]
    study_case.Activate()
    if verbose: print(f'Successfully activated study case {study_case_name}.')
    #
    load_flow = app.GetFromStudyCase('ComLdf')
    err = load_flow.Execute()
    if err:
        raise Exception(f'Cannot run load flow')
    if verbose: print(f'Successfully run load flow.')
    #
    results = {key: {} for key in ('generators','buses','loads')}
    
    Ptot, Qtot = 0, 0
    for gen in generators:
        pq = [gen.GetAttribute(f'm:{c}sum:bus1') for c in 'PQ']
        results['generators'][gen.loc_name] = {
            'P': pq[0],
            'Q': pq[1],
            'I': gen.GetAttribute('m:I:bus1'),
            'V': gen.GetAttribute('m:U1:bus1'),    # line-to-ground voltage
            'Vl': gen.GetAttribute('m:U1l:bus1')  # line-to-line voltage
        }
        Ptot += pq[0]
        Qtot += pq[1]
    results['generators']['Ptot'] = Ptot
    results['generators']['Qtot'] = Qtot
        
    Ptot, Qtot = 0, 0
    for load in loads:
        pq = [load.GetAttribute(f'm:{c}sum:bus1') for c in 'PQ']
        results['loads'][load.loc_name] = {
            'P': pq[0],
            'Q': pq[1],
            'I': load.GetAttribute('m:I:bus1'),
            'V': load.GetAttribute('m:U1:bus1'),    # line-to-ground voltage
            'Vl': load.GetAttribute('m:U1l:bus1'),  # line-to-line voltage
        }
        Ptot += pq[0]
        Qtot += pq[1]
    results['loads']['Ptot'] = Ptot
    results['loads']['Qtot'] = Qtot
    
    for bus in buses:
        results['buses'][bus.loc_name] = {
            'voltage': bus.GetAttribute('m:u'),
            'V': bus.GetAttribute('m:U'),
            'Vl': bus.GetAttribute('m:Ul')
        }

    return results

def print_load_flow(results):
    print('\n===== Generators =====')
    for name,data in results['generators'].items():
        if name not in ('Ptot','Qtot'):
            print(f'{name}: P = {data["P"]:7.2f} MW, Q = {data["Q"]:6.2f} MVAR, ' + 
                  f'I = {data["I"]:6.3f} kA, V = {data["V"]:6.3f} kV.')
    print(f'Total P = {results["generators"]["Ptot"]*1e-3:5.2f} GW, total Q = {results["generators"]["Qtot"]*1e-3:5.2f} GVAR')

    print('\n======= Buses ========')
    for name,data in results['buses'].items():
        print(f'{name}: voltage = {data["voltage"]:5.3f} pu, V = {data["Vl"]:7.3f} kV.')
        
    print('\n======= Loads ========')
    for name,data in results['loads'].items():
        if name not in ('Ptot','Qtot'):
            print(f'{name}: P = {data["P"]:7.2f} MW, Q = {data["Q"]:6.2f} MVAR, ' + 
                  f'I = {data["I"]:6.3f} kA, V = {data["V"]:8.3f} kV.')
    print(f'Total P = {results["loads"]["Ptot"]*1e-3:5.2f} GW, total Q = {results["loads"]["Qtot"]*1e-3:5.2f} GVAR')

In [ ]:
defaults = {
    'pgini':  [None, 163, 85],
    'plini': [125, 90, 100],
    'qlini': [50, 30, 35]
}
lf_res = {}
LAMBDA = np.r_[0 : 31] / 100
for coeff in LAMBDA:
    for generator, pgini in zip(generators, defaults['pgini']):
        if pgini is not None:
            generator.pgini = (1 + coeff) * pgini
    for load, plini, qlini in zip(loads, defaults['plini'], defaults['qlini']):
        load.plini = (1 + coeff) * plini
        load.qlini = (1 + coeff) * qlini
    lf_res[coeff] = run_load_flow(project_folders['study'], generators, loads, buses)
pickle.dump(lf_res, open('WSCC_9_bus_load_flow_digsilent_overload.pkl','wb'))

### Reset the parameters to their default values
for generator, pgini in zip(generators, defaults['pgini']):
    if pgini is not None:
        generator.pgini = pgini
for load, plini, qlini in zip(loads, defaults['plini'], defaults['qlini']):
    load.plini = plini
    load.qlini = qlini

In [ ]:
lf_res = run_load_flow(project_folders['study'], generators, loads, buses)
pickle.dump(lf_res, open('WSCC_9_bus_load_flow_digsilent.pkl','wb'))
print_load_flow(lf_res)